In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('./Sets/911.csv')

# Datetime, Offset, Timedelta, Rolling

https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html

Esta libreta está DENSA.

## Datetime

Checamos el tipo de columna donde debería de haber fechas.

In [4]:
df.timeStamp.dtypes

dtype('O')

No hay fechas, hay strings!

In [5]:
df.timeStamp

0        2015-12-10 17:40:00
1        2015-12-10 17:40:00
2        2015-12-10 17:40:00
3        2015-12-10 17:40:01
4        2015-12-10 17:40:01
                ...         
99487    2016-08-24 11:06:00
99488    2016-08-24 11:07:02
99489    2016-08-24 11:12:00
99490    2016-08-24 11:17:01
99491    2016-08-24 11:17:02
Name: timeStamp, Length: 99492, dtype: object

Con pd.to_datetime convertimos a string

Ojo con dar el formato correcto. Revisa esta liga para conocer los formatos: https://strftime.org/

In [6]:
fechas = pd.to_datetime(df.timeStamp, format = '%Y-%m-%d %H:%M:%S')
fechas

0       2015-12-10 17:40:00
1       2015-12-10 17:40:00
2       2015-12-10 17:40:00
3       2015-12-10 17:40:01
4       2015-12-10 17:40:01
                ...        
99487   2016-08-24 11:06:00
99488   2016-08-24 11:07:02
99489   2016-08-24 11:12:00
99490   2016-08-24 11:17:01
99491   2016-08-24 11:17:02
Name: timeStamp, Length: 99492, dtype: datetime64[ns]

Checamos dtype para confirmar que efectivamente tenemos una fecha

In [7]:
fechas.dtypes

dtype('<M8[ns]')

Una fecha tiene los atributos siguientes:
- Año
- Mes
- Día
- Hora
- Minuto
- Segundo
- Fracciones de segundo

In [8]:
fechas[0].year, fechas[0].month, fechas[0].day, fechas[0].hour, fechas[0].minute, fechas[0].second 

(2015, 12, 10, 17, 40, 0)

Si quieres aplicar un método de datetime a todo el dataset necesitas aplicar el .dt primero

In [9]:
fechas.dt.year

0        2015
1        2015
2        2015
3        2015
4        2015
         ... 
99487    2016
99488    2016
99489    2016
99490    2016
99491    2016
Name: timeStamp, Length: 99492, dtype: int64

Además hay algunos métodos y otros atribuos útiles

In [9]:
fechas.dt.month_name()

0        December
1        December
2        December
3        December
4        December
           ...   
99487      August
99488      August
99489      August
99490      August
99491      August
Name: timeStamp, Length: 99492, dtype: object

In [10]:
fechas.dt.day_name()

0         Thursday
1         Thursday
2         Thursday
3         Thursday
4         Thursday
           ...    
99487    Wednesday
99488    Wednesday
99489    Wednesday
99490    Wednesday
99491    Wednesday
Name: timeStamp, Length: 99492, dtype: object

Lunes es 0  
Domingo es 6

In [11]:
fechas.dt.weekday

0        3
1        3
2        3
3        3
4        3
        ..
99487    2
99488    2
99489    2
99490    2
99491    2
Name: timeStamp, Length: 99492, dtype: int64

In [12]:
fechas.dt.dayofyear

0        344
1        344
2        344
3        344
4        344
        ... 
99487    237
99488    237
99489    237
99490    237
99491    237
Name: timeStamp, Length: 99492, dtype: int64

In [13]:
fechas.dt.quarter

0        4
1        4
2        4
3        4
4        4
        ..
99487    3
99488    3
99489    3
99490    3
99491    3
Name: timeStamp, Length: 99492, dtype: int64

In [14]:
fechas.dt.days_in_month

0        31
1        31
2        31
3        31
4        31
         ..
99487    31
99488    31
99489    31
99490    31
99491    31
Name: timeStamp, Length: 99492, dtype: int64

**En formato ISO lunes es 1, domingo es 7**

**isocalendar puede fallar según la versión de pandas**

In [15]:
fechas.dt.isocalendar()

,year,week,day
0,2015,50,4
1,2015,50,4
2,2015,50,4
3,2015,50,4
4,2015,50,4
...,...,...,...
99487,2016,34,3
99488,2016,34,3
99489,2016,34,3
99490,2016,34,3


In [16]:
fechas.dt.isocalendar().week

0        50
1        50
2        50
3        50
4        50
         ..
99487    34
99488    34
99489    34
99490    34
99491    34
Name: week, Length: 99492, dtype: UInt32

Puedes usar strftime para presentar la fecha en el formato que quieras (checa la liga a strftime que puse al inicio de la libreta)

In [17]:
fechas.dt.strftime('%d de %b del %Y a las %H con %M minutos')

0        10 de Dec del 2015 a las 17 con 40 minutos
1        10 de Dec del 2015 a las 17 con 40 minutos
2        10 de Dec del 2015 a las 17 con 40 minutos
3        10 de Dec del 2015 a las 17 con 40 minutos
4        10 de Dec del 2015 a las 17 con 40 minutos
                            ...                    
99487    24 de Aug del 2016 a las 11 con 06 minutos
99488    24 de Aug del 2016 a las 11 con 07 minutos
99489    24 de Aug del 2016 a las 11 con 12 minutos
99490    24 de Aug del 2016 a las 11 con 17 minutos
99491    24 de Aug del 2016 a las 11 con 17 minutos
Name: timeStamp, Length: 99492, dtype: object

**resample**

Puedes homogenizar la frecuencia de tus entradas mediante un resample. Por ejemplo, en vez de tener 100 entradas para un día, quiero sólo el promedio de todas las entradas de ese día. Para esto primero necesitamos poner la fecha como índice y agregar todos los resultados de un día en un sólo número. Puedes checar todas las frecuencias por las que puedes resamplear tus fechas aquí: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases

In [18]:
df_fechas = df.copy()
df_fechas['fechas'] = fechas
df_fechas.set_index('fechas',inplace=True)

In [19]:
df_fechas.head(3)

,lat,lng,desc,zip,title,timeStamp,twp,addr,e
fechas,,,,,,,,,
2015-12-10 17:40:00,40.297876,-75.581294,REINDEER CT & DEAD END; NEW HANOVER; Station ...,19525.0,EMS: BACK PAINS/INJURY,2015-12-10 17:40:00,NEW HANOVER,REINDEER CT & DEAD END,1
2015-12-10 17:40:00,40.258061,-75.264680,BRIAR PATH & WHITEMARSH LN; HATFIELD TOWNSHIP...,19446.0,EMS: DIABETIC EMERGENCY,2015-12-10 17:40:00,HATFIELD TOWNSHIP,BRIAR PATH & WHITEMARSH LN,1
2015-12-10 17:40:00,40.121182,-75.351975,HAWS AVE; NORRISTOWN; 2015-12-10 @ 14:39:21-St...,19401.0,Fire: GAS-ODOR/LEAK,2015-12-10 17:40:00,NORRISTOWN,HAWS AVE,1


Notemos que el index tiene un freq = None!

In [20]:
df_fechas.index

DatetimeIndex(['2015-12-10 17:40:00', '2015-12-10 17:40:00',
               '2015-12-10 17:40:00', '2015-12-10 17:40:01',
               '2015-12-10 17:40:01', '2015-12-10 17:40:01',
               '2015-12-10 17:40:01', '2015-12-10 17:40:01',
               '2015-12-10 17:40:01', '2015-12-10 17:40:01',
               ...
               '2016-08-24 10:52:03', '2016-08-24 10:57:00',
               '2016-08-24 10:57:00', '2016-08-24 10:57:01',
               '2016-08-24 11:02:02', '2016-08-24 11:06:00',
               '2016-08-24 11:07:02', '2016-08-24 11:12:00',
               '2016-08-24 11:17:01', '2016-08-24 11:17:02'],
              dtype='datetime64[ns]', name='fechas', length=99492, freq=None)

Y ahora con un resample reajustamos todo a una frecuencia diaria.

In [21]:
df_fechas.resample('d').mean()

,lat,lng,zip,e
fechas,,,,
2015-12-10,40.163933,-75.341467,19293.700000,1
2015-12-11,40.164502,-75.312044,19215.666667,1
2015-12-12,40.156301,-75.318920,19271.543544,1
2015-12-13,40.156752,-75.307261,19219.217857,1
2015-12-14,40.157897,-75.311976,19228.113695,1
...,...,...,...,...
2016-08-20,40.161065,-75.335873,19245.770609,1
2016-08-21,40.163826,-75.333098,19216.872910,1
2016-08-22,40.160923,-75.305836,19218.928571,1


Notemos que el index tiene una freq diaria!

In [22]:
df_fechas.resample('d').mean().index

DatetimeIndex(['2015-12-10', '2015-12-11', '2015-12-12', '2015-12-13',
               '2015-12-14', '2015-12-15', '2015-12-16', '2015-12-17',
               '2015-12-18', '2015-12-19',
               ...
               '2016-08-15', '2016-08-16', '2016-08-17', '2016-08-18',
               '2016-08-19', '2016-08-20', '2016-08-21', '2016-08-22',
               '2016-08-23', '2016-08-24'],
              dtype='datetime64[ns]', name='fechas', length=259, freq='D')

Podemos usar multiplicadores como 3, referenciando a el promedio de cada 3 días

In [23]:
df_fechas.resample('3d').mean()

,lat,lng,zip,e
fechas,,,,
2015-12-10,40.160815,-75.318777,19250.144909,1
2015-12-13,40.155474,-75.305738,19220.411652,1
2015-12-16,40.161286,-75.321510,19243.895431,1
2015-12-19,40.164556,-75.318909,19240.855464,1
2015-12-22,40.159656,-75.310632,19226.786831,1
...,...,...,...,...
2016-08-12,40.158638,-75.315371,19235.103755,1
2016-08-15,40.153484,-75.305107,19220.474544,1
2016-08-18,40.153531,-75.315151,19238.576763,1


El index se da cuenta de este cambio!

In [24]:
df_fechas.resample('3d').mean().index

DatetimeIndex(['2015-12-10', '2015-12-13', '2015-12-16', '2015-12-19',
               '2015-12-22', '2015-12-25', '2015-12-28', '2015-12-31',
               '2016-01-03', '2016-01-06', '2016-01-09', '2016-01-12',
               '2016-01-15', '2016-01-18', '2016-01-21', '2016-01-24',
               '2016-01-27', '2016-01-30', '2016-02-02', '2016-02-05',
               '2016-02-08', '2016-02-11', '2016-02-14', '2016-02-17',
               '2016-02-20', '2016-02-23', '2016-02-26', '2016-02-29',
               '2016-03-03', '2016-03-06', '2016-03-09', '2016-03-12',
               '2016-03-15', '2016-03-18', '2016-03-21', '2016-03-24',
               '2016-03-27', '2016-03-30', '2016-04-02', '2016-04-05',
               '2016-04-08', '2016-04-11', '2016-04-14', '2016-04-17',
               '2016-04-20', '2016-04-23', '2016-04-26', '2016-04-29',
               '2016-05-02', '2016-05-05', '2016-05-08', '2016-05-11',
               '2016-05-14', '2016-05-17', '2016-05-20', '2016-05-23',
      

OJO si por ejemplo, la fecha 2015-12-13 (Año - mes - día) no existiera en el dataset por lo que no hay nada que promediar, resample marcaría un nan  

Además si resampleamos a una frecuencia superior a nuestros datos (yo registro cada día pero quiero ver los valores por hora) vamos a tener nans. Es imposible obtener los datos exactos por hora si no los registro, pero mediante interpolación puedo hacer una aproximación aritmética (que probablemente esté mal de todos modos).

Aquí estamos viendo las primeras 5 entradas a una resolucion de microsegundo. Como no tenermos datos entre microsegundos pandas nos da nans. Para poner valores ahí sólo tenemos que correr interpolate

In [25]:
df_fechas.iloc[0:5].resample('us').mean()

,lat,lng,zip,e
fechas,,,,
2015-12-10 17:40:00.000000,40.225706,-75.399316,19457.333333,1.0
2015-12-10 17:40:00.000001,NaN,NaN,NaN,NaN
2015-12-10 17:40:00.000002,NaN,NaN,NaN,NaN
2015-12-10 17:40:00.000003,NaN,NaN,NaN,NaN
2015-12-10 17:40:00.000004,NaN,NaN,NaN,NaN
...,...,...,...,...
2015-12-10 17:40:00.999996,NaN,NaN,NaN,NaN
2015-12-10 17:40:00.999997,NaN,NaN,NaN,NaN
2015-12-10 17:40:00.999998,NaN,NaN,NaN,NaN


https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.interpolate.html  para ver métodos diferentes al lineal para llenar la información

In [26]:
df_fechas.iloc[0:5].resample('us').mean().interpolate(method='linear')

,lat,lng,zip,e
fechas,,,,
2015-12-10 17:40:00.000000,40.225706,-75.399316,19457.333333,1.0
2015-12-10 17:40:00.000001,40.225706,-75.399316,19457.333277,1.0
2015-12-10 17:40:00.000002,40.225706,-75.399316,19457.333221,1.0
2015-12-10 17:40:00.000003,40.225706,-75.399316,19457.333164,1.0
2015-12-10 17:40:00.000004,40.225706,-75.399316,19457.333108,1.0
...,...,...,...,...
2015-12-10 17:40:00.999996,40.183823,-75.473431,19401.000225,1.0
2015-12-10 17:40:00.999997,40.183823,-75.473431,19401.000169,1.0
2015-12-10 17:40:00.999998,40.183823,-75.473431,19401.000113,1.0


Además podemos hacer slices de las fechas como índice.  
Ambos límites serán **inclusivos**.

In [27]:
df_fechas.loc['2015-12-12':'2015-12-13'].head(3)

,lat,lng,desc,zip,title,timeStamp,twp,addr,e
fechas,,,,,,,,,
2015-12-12 00:02:00,40.235373,-75.224751,STUMP RD & HORSHAM RD; MONTGOMERY; Station 34...,19454.0,EMS: HEAD INJURY,2015-12-12 00:02:00,MONTGOMERY,STUMP RD & HORSHAM RD,1
2015-12-12 00:17:00,40.011767,-75.294673,MONTGOMERY AVE & WOODSIDE RD; LOWER MERION; S...,19041.0,EMS: CARDIAC EMERGENCY,2015-12-12 00:17:00,LOWER MERION,MONTGOMERY AVE & WOODSIDE RD,1
2015-12-12 00:27:01,40.124147,-75.060930,RED LION RD & MURRAY AVE; LOWER MORELAND; Sta...,19006.0,EMS: EMS SPECIAL SERVICE,2015-12-12 00:27:01,LOWER MORELAND,RED LION RD & MURRAY AVE,1


Puedes incluso dar strings parciales y te regresará los índices correspondientes (en este caso, todo lo que haya ocurrido en diciembre 2015)

In [28]:
df_fechas.loc['2015 - 12'].head(3)

,lat,lng,desc,zip,title,timeStamp,twp,addr,e
fechas,,,,,,,,,
2015-12-10 17:40:00,40.297876,-75.581294,REINDEER CT & DEAD END; NEW HANOVER; Station ...,19525.0,EMS: BACK PAINS/INJURY,2015-12-10 17:40:00,NEW HANOVER,REINDEER CT & DEAD END,1
2015-12-10 17:40:00,40.258061,-75.264680,BRIAR PATH & WHITEMARSH LN; HATFIELD TOWNSHIP...,19446.0,EMS: DIABETIC EMERGENCY,2015-12-10 17:40:00,HATFIELD TOWNSHIP,BRIAR PATH & WHITEMARSH LN,1
2015-12-10 17:40:00,40.121182,-75.351975,HAWS AVE; NORRISTOWN; 2015-12-10 @ 14:39:21-St...,19401.0,Fire: GAS-ODOR/LEAK,2015-12-10 17:40:00,NORRISTOWN,HAWS AVE,1


En vez de loc puedes usar:

**at_time, between_time**

In [29]:
df_fechas.at_time('22:01').head()

,lat,lng,desc,zip,title,timeStamp,twp,addr,e
fechas,,,,,,,,,
2015-12-17 22:01:00,40.291967,-75.602027,BUCHERT RD & DONALD DR; NEW HANOVER; Station ...,19464.0,EMS: ASSAULT VICTIM,2015-12-17 22:01:00,NEW HANOVER,BUCHERT RD & DONALD DR,1
2015-12-23 22:01:00,40.250721,-75.675935,GLASGOW ST & SCHOOL LN; WEST POTTSGROVE; 2015-...,19464.0,Traffic: VEHICLE ACCIDENT -,2015-12-23 22:01:00,WEST POTTSGROVE,GLASGOW ST & SCHOOL LN,1
2015-12-24 22:01:00,40.124807,-75.340743,MARKLEY ST & STERIGERE ST; NORRISTOWN; Statio...,19401.0,EMS: UNKNOWN MEDICAL EMERGENCY,2015-12-24 22:01:00,NORRISTOWN,MARKLEY ST & STERIGERE ST,1
2015-12-24 22:01:00,40.112203,-75.422090,NESTER DR & EVAN DR; LOWER PROVIDENCE; 2015-12...,19403.0,Fire: CARBON MONOXIDE DETECTOR,2015-12-24 22:01:00,LOWER PROVIDENCE,NESTER DR & EVAN DR,1
2016-01-15 22:01:00,40.229008,-75.387852,NO LOCATION - NEIGHBORING COUNTY; CHESTER COU...,NaN,EMS: SUBJECT IN PAIN,2016-01-15 22:01:00,CHESTER COUNTY,NO LOCATION - NEIGHBORING COUNTY,1


In [30]:
df_fechas.between_time('22:01','22:05')

,lat,lng,desc,zip,title,timeStamp,twp,addr,e
fechas,,,,,,,,,
2015-12-10 22:01:01,40.276304,-75.565024,SWAMP PIKE & SANATOGA RD; NEW HANOVER; 2015-12...,19525.0,Traffic: DISABLED VEHICLE -,2015-12-10 22:01:01,NEW HANOVER,SWAMP PIKE & SANATOGA RD,1
2015-12-10 22:02:01,40.095320,-75.193482,CHELTENHAM AVE & DELPHINE RD; SPRINGFIELD; 201...,19038.0,Traffic: VEHICLE ACCIDENT -,2015-12-10 22:02:01,SPRINGFIELD,CHELTENHAM AVE & DELPHINE RD,1
2015-12-11 22:02:01,40.087285,-75.293087,BUTLER PIKE & E NORTH LN; WHITEMARSH; 2015-12-...,19428.0,Traffic: DISABLED VEHICLE -,2015-12-11 22:02:01,WHITEMARSH,BUTLER PIKE & E NORTH LN,1
2015-12-12 22:02:01,40.172671,-75.402652,GERMANTOWN PIKE; WORCESTER; Station 322; 2015...,NaN,EMS: VEHICLE ACCIDENT,2015-12-12 22:02:01,WORCESTER,GERMANTOWN PIKE,1
2015-12-12 22:02:01,40.087858,-75.254296,GERMANTOWN PIKE & WESTAWAY DR; WHITEMARSH; 201...,19444.0,Fire: UNKNOWN TYPE FIRE,2015-12-12 22:02:01,WHITEMARSH,GERMANTOWN PIKE & WESTAWAY DR,1
...,...,...,...,...,...,...,...,...,...
2016-08-17 22:02:01,40.289027,-75.399590,MAIN ST & OLD SUMNEYTOWN PIKE; LOWER SALFORD;...,19438.0,EMS: RESPIRATORY EMERGENCY,2016-08-17 22:02:01,LOWER SALFORD,MAIN ST & OLD SUMNEYTOWN PIKE,1
2016-08-18 22:02:02,40.176082,-75.123128,BLAIR MILL RD & HORSHAM RD; UPPER MORELAND; 20...,19044.0,Traffic: VEHICLE ACCIDENT -,2016-08-18 22:02:02,UPPER MORELAND,BLAIR MILL RD & HORSHAM RD,1
2016-08-18 22:02:02,40.087568,-75.405559,GULPH RD & RAMP I76 EB TO N GULPH RD; UPPER ME...,NaN,Traffic: VEHICLE ACCIDENT -,2016-08-18 22:02:02,UPPER MERION,GULPH RD & RAMP I76 EB TO N GULPH RD,1


Si el índice es un datetime puedes usar los mismos métodos con este que como si fuera una columna

In [31]:
df_fechas.index.hour

Int64Index([17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
            ...
            10, 10, 10, 10, 11, 11, 11, 11, 11, 11],
           dtype='int64', name='fechas', length=99492)

In [32]:
df_fechas.index.month

Int64Index([12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
            ...
             8,  8,  8,  8,  8,  8,  8,  8,  8,  8],
           dtype='int64', name='fechas', length=99492)

In [33]:
df_fechas.index.day_name()

Index(['Thursday', 'Thursday', 'Thursday', 'Thursday', 'Thursday', 'Thursday',
       'Thursday', 'Thursday', 'Thursday', 'Thursday',
       ...
       'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday',
       'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday'],
      dtype='object', name='fechas', length=99492)

Un truco ligeramente avanzado, puedes obtener el promedio por día de todo tu dataset con este groupby.

In [34]:
df_fechas.groupby(by = df_fechas.index.month_name()).mean()

,lat,lng,zip,e
fechas,,,,
April,40.158850,-75.315631,19235.209096,1
August,40.157054,-75.313496,19230.362615,1
December,40.161100,-75.315802,19234.623136,1
February,40.160668,-75.321789,19234.750453,1
January,40.163586,-75.321511,19244.238397,1
July,40.158710,-75.316873,19235.601335,1
June,40.156001,-75.315802,19246.380729,1
March,40.160263,-75.318207,19239.100974,1
May,40.159013,-75.316190,19235.053187,1


In [35]:
df_fechas.groupby(by = df_fechas.index.day_name()).mean()

,lat,lng,zip,e
fechas,,,,
Friday,40.158524,-75.316152,19236.375029,1
Monday,40.158085,-75.312809,19237.572201,1
Saturday,40.161255,-75.325025,19243.702721,1
Sunday,40.162670,-75.326960,19238.662625,1
Thursday,40.158250,-75.317889,19242.754531,1
Tuesday,40.158984,-75.312341,19231.361854,1
Wednesday,40.159628,-75.313642,19234.273139,1


**Daterange**  
Por si quieres crear un rango de fechas  
Ojo con el parámetro freq. Experimentar con esto hará más fácil la siguiente sección

Checa las frecuencias posibles aquí: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-offset-aliases

In [36]:
pd.date_range('5/10/2020','10/10/2020',freq='Min')

DatetimeIndex(['2020-05-10 00:00:00', '2020-05-10 00:01:00',
               '2020-05-10 00:02:00', '2020-05-10 00:03:00',
               '2020-05-10 00:04:00', '2020-05-10 00:05:00',
               '2020-05-10 00:06:00', '2020-05-10 00:07:00',
               '2020-05-10 00:08:00', '2020-05-10 00:09:00',
               ...
               '2020-10-09 23:51:00', '2020-10-09 23:52:00',
               '2020-10-09 23:53:00', '2020-10-09 23:54:00',
               '2020-10-09 23:55:00', '2020-10-09 23:56:00',
               '2020-10-09 23:57:00', '2020-10-09 23:58:00',
               '2020-10-09 23:59:00', '2020-10-10 00:00:00'],
              dtype='datetime64[ns]', length=220321, freq='T')

In [37]:
pd.date_range('5/10/2020','10/10/2020',freq='D')

DatetimeIndex(['2020-05-10', '2020-05-11', '2020-05-12', '2020-05-13',
               '2020-05-14', '2020-05-15', '2020-05-16', '2020-05-17',
               '2020-05-18', '2020-05-19',
               ...
               '2020-10-01', '2020-10-02', '2020-10-03', '2020-10-04',
               '2020-10-05', '2020-10-06', '2020-10-07', '2020-10-08',
               '2020-10-09', '2020-10-10'],
              dtype='datetime64[ns]', length=154, freq='D')

In [38]:
pd.date_range('5/10/2020','10/10/2020',freq='W')

DatetimeIndex(['2020-05-10', '2020-05-17', '2020-05-24', '2020-05-31',
               '2020-06-07', '2020-06-14', '2020-06-21', '2020-06-28',
               '2020-07-05', '2020-07-12', '2020-07-19', '2020-07-26',
               '2020-08-02', '2020-08-09', '2020-08-16', '2020-08-23',
               '2020-08-30', '2020-09-06', '2020-09-13', '2020-09-20',
               '2020-09-27', '2020-10-04'],
              dtype='datetime64[ns]', freq='W-SUN')

Diez semanas. Por qué no?

In [39]:
pd.date_range('5/10/2020','10/10/2020',freq='10W')

DatetimeIndex(['2020-05-10', '2020-07-19', '2020-09-27'], dtype='datetime64[ns]', freq='10W-SUN')

In [40]:
pd.date_range('5/10/2020','10/10/2020',freq='M')

DatetimeIndex(['2020-05-31', '2020-06-30', '2020-07-31', '2020-08-31',
               '2020-09-30'],
              dtype='datetime64[ns]', freq='M')

In [41]:
pd.date_range('5/10/2020','10/10/2020',freq='Q')

DatetimeIndex(['2020-06-30', '2020-09-30'], dtype='datetime64[ns]', freq='Q-DEC')

In [42]:
pd.date_range('5/10/2020','10/10/2020',freq='Y')

DatetimeIndex([], dtype='datetime64[ns]', freq='A-DEC')

Por inicio y fin

In [43]:
pd.date_range(start='dec 2020', periods = 15, freq = 'M')

DatetimeIndex(['2020-12-31', '2021-01-31', '2021-02-28', '2021-03-31',
               '2021-04-30', '2021-05-31', '2021-06-30', '2021-07-31',
               '2021-08-31', '2021-09-30', '2021-10-31', '2021-11-30',
               '2021-12-31', '2022-01-31', '2022-02-28'],
              dtype='datetime64[ns]', freq='M')

In [44]:
pd.date_range(end='feb 2022', periods = 15, freq = 'M')

DatetimeIndex(['2020-11-30', '2020-12-31', '2021-01-31', '2021-02-28',
               '2021-03-31', '2021-04-30', '2021-05-31', '2021-06-30',
               '2021-07-31', '2021-08-31', '2021-09-30', '2021-10-31',
               '2021-11-30', '2021-12-31', '2022-01-31'],
              dtype='datetime64[ns]', freq='M')

In [45]:
pd.date_range(end='jan 2021',periods=4,freq='D')

DatetimeIndex(['2020-12-29', '2020-12-30', '2020-12-31', '2021-01-01'], dtype='datetime64[ns]', freq='D')

**Asfreq**

In [46]:
df_fechas.iloc[0:10,:]

,lat,lng,desc,zip,title,timeStamp,twp,addr,e
fechas,,,,,,,,,
2015-12-10 17:40:00,40.297876,-75.581294,REINDEER CT & DEAD END; NEW HANOVER; Station ...,19525.0,EMS: BACK PAINS/INJURY,2015-12-10 17:40:00,NEW HANOVER,REINDEER CT & DEAD END,1
2015-12-10 17:40:00,40.258061,-75.264680,BRIAR PATH & WHITEMARSH LN; HATFIELD TOWNSHIP...,19446.0,EMS: DIABETIC EMERGENCY,2015-12-10 17:40:00,HATFIELD TOWNSHIP,BRIAR PATH & WHITEMARSH LN,1
2015-12-10 17:40:00,40.121182,-75.351975,HAWS AVE; NORRISTOWN; 2015-12-10 @ 14:39:21-St...,19401.0,Fire: GAS-ODOR/LEAK,2015-12-10 17:40:00,NORRISTOWN,HAWS AVE,1
2015-12-10 17:40:01,40.116153,-75.343513,AIRY ST & SWEDE ST; NORRISTOWN; Station 308A;...,19401.0,EMS: CARDIAC EMERGENCY,2015-12-10 17:40:01,NORRISTOWN,AIRY ST & SWEDE ST,1
2015-12-10 17:40:01,40.251492,-75.603350,CHERRYWOOD CT & DEAD END; LOWER POTTSGROVE; S...,NaN,EMS: DIZZINESS,2015-12-10 17:40:01,LOWER POTTSGROVE,CHERRYWOOD CT & DEAD END,1
2015-12-10 17:40:01,40.253473,-75.283245,CANNON AVE & W 9TH ST; LANSDALE; Station 345;...,19446.0,EMS: HEAD INJURY,2015-12-10 17:40:01,LANSDALE,CANNON AVE & W 9TH ST,1
2015-12-10 17:40:01,40.182111,-75.127795,LAUREL AVE & OAKDALE AVE; HORSHAM; Station 35...,19044.0,EMS: NAUSEA/VOMITING,2015-12-10 17:40:01,HORSHAM,LAUREL AVE & OAKDALE AVE,1
2015-12-10 17:40:01,40.217286,-75.405182,COLLEGEVILLE RD & LYWISKI RD; SKIPPACK; Stati...,19426.0,EMS: RESPIRATORY EMERGENCY,2015-12-10 17:40:01,SKIPPACK,COLLEGEVILLE RD & LYWISKI RD,1
2015-12-10 17:40:01,40.289027,-75.399590,MAIN ST & OLD SUMNEYTOWN PIKE; LOWER SALFORD;...,19438.0,EMS: SYNCOPAL EPISODE,2015-12-10 17:40:01,LOWER SALFORD,MAIN ST & OLD SUMNEYTOWN PIKE,1


Asfreq funciona similar a resample en el sentido de que reestructura la frecuencia de las fechas.

Por ejemplo, resampleamos las fechas para tener una frecuencia diaria con el promedio de cada valor

In [47]:
resamp = df_fechas.resample('D').mean()
resamp

,lat,lng,zip,e
fechas,,,,
2015-12-10,40.163933,-75.341467,19293.700000,1
2015-12-11,40.164502,-75.312044,19215.666667,1
2015-12-12,40.156301,-75.318920,19271.543544,1
2015-12-13,40.156752,-75.307261,19219.217857,1
2015-12-14,40.157897,-75.311976,19228.113695,1
...,...,...,...,...
2016-08-20,40.161065,-75.335873,19245.770609,1
2016-08-21,40.163826,-75.333098,19216.872910,1
2016-08-22,40.160923,-75.305836,19218.928571,1


Luego usamos asfreq para obtener sólo los valores al final de cada mes (sin andar agregando por promedio o algo así)

In [48]:
resamp.asfreq('M')

,lat,lng,zip,e
fechas,,,,
2015-12-31,40.156669,-75.307671,19210.691450,1
2016-01-31,40.167765,-75.340673,19257.458015,1
2016-02-29,40.158565,-75.310156,19215.938312,1
2016-03-31,40.164648,-75.321494,19229.304075,1
2016-04-30,40.163442,-75.341752,19284.231343,1
2016-05-31,40.151441,-75.306211,19219.256267,1
2016-06-30,40.166293,-75.327071,19247.008671,1
2016-07-31,40.155001,-75.323931,19217.877023,1


Si te fijas no tuvimos que promediar ni nada. Asfreq te regresa exclusivamente el valor en el periodo que le pediste. En este caso nos regresó el valor del último día de cada mes

Sería equivalente a pedir un resample y .last() como método de agregacin

OJO: asfreq sólo funciona cuando los índices (fechas) son únicos! Su gran ventaja es que puede usar method = 'ffill' o 'bfill' para rellenar nans en caso de que se creen con el comando.

In [49]:
resamp.resample('M').last()

,lat,lng,zip,e
fechas,,,,
2015-12-31,40.156669,-75.307671,19210.691450,1
2016-01-31,40.167765,-75.340673,19257.458015,1
2016-02-29,40.158565,-75.310156,19215.938312,1
2016-03-31,40.164648,-75.321494,19229.304075,1
2016-04-30,40.163442,-75.341752,19284.231343,1
2016-05-31,40.151441,-75.306211,19219.256267,1
2016-06-30,40.166293,-75.327071,19247.008671,1
2016-07-31,40.155001,-75.323931,19217.877023,1
2016-08-31,40.160704,-75.310376,19231.415094,1


**duplicates**

Puedes usar duplicated sober el índice para obtener los puntos donde las fechas se repiten.

In [65]:
df_fechas.loc[df_fechas.index.duplicated(keep = False),:]

,lat,lng,desc,zip,title,timeStamp,twp,addr,e
fechas,,,,,,,,,
2015-12-10 17:40:00,40.297876,-75.581294,REINDEER CT & DEAD END; NEW HANOVER; Station ...,19525.0,EMS: BACK PAINS/INJURY,2015-12-10 17:40:00,NEW HANOVER,REINDEER CT & DEAD END,1
2015-12-10 17:40:00,40.258061,-75.264680,BRIAR PATH & WHITEMARSH LN; HATFIELD TOWNSHIP...,19446.0,EMS: DIABETIC EMERGENCY,2015-12-10 17:40:00,HATFIELD TOWNSHIP,BRIAR PATH & WHITEMARSH LN,1
2015-12-10 17:40:00,40.121182,-75.351975,HAWS AVE; NORRISTOWN; 2015-12-10 @ 14:39:21-St...,19401.0,Fire: GAS-ODOR/LEAK,2015-12-10 17:40:00,NORRISTOWN,HAWS AVE,1
2015-12-10 17:40:01,40.116153,-75.343513,AIRY ST & SWEDE ST; NORRISTOWN; Station 308A;...,19401.0,EMS: CARDIAC EMERGENCY,2015-12-10 17:40:01,NORRISTOWN,AIRY ST & SWEDE ST,1
2015-12-10 17:40:01,40.251492,-75.603350,CHERRYWOOD CT & DEAD END; LOWER POTTSGROVE; S...,NaN,EMS: DIZZINESS,2015-12-10 17:40:01,LOWER POTTSGROVE,CHERRYWOOD CT & DEAD END,1
...,...,...,...,...,...,...,...,...,...
2016-08-24 10:52:01,40.221227,-75.288737,SUMNEYTOWN PIKE & RR OVERPASS; UPPER GWYNEDD;...,NaN,EMS: CARDIAC EMERGENCY,2016-08-24 10:52:01,UPPER GWYNEDD,SUMNEYTOWN PIKE & RR OVERPASS,1
2016-08-24 10:52:01,40.221227,-75.288737,SUMNEYTOWN PIKE & RR OVERPASS; UPPER GWYNEDD;...,NaN,EMS: DIABETIC EMERGENCY,2016-08-24 10:52:01,UPPER GWYNEDD,SUMNEYTOWN PIKE & RR OVERPASS,1
2016-08-24 10:52:01,40.221227,-75.288737,SUMNEYTOWN PIKE & RR OVERPASS; UPPER GWYNEDD;...,NaN,EMS: DIZZINESS,2016-08-24 10:52:01,UPPER GWYNEDD,SUMNEYTOWN PIKE & RR OVERPASS,1


## Offset

Por ninguna razón en especial quiero agregar 10 meses a todas mis fechas, si trato de hacer esta operación tendré un error

In [51]:
fechas + 10

TypeError: Addition/subtraction of integers and integer-arrays with DatetimeArray is no longer supported.  Instead of adding/subtracting `n`, use `n * obj.freq`

La manera correcta sería con un offset

In [52]:
pd.DateOffset(months=10)

<DateOffset: months=10>

In [53]:
fechas + pd.DateOffset(months=10)

0       2016-10-10 17:40:00
1       2016-10-10 17:40:00
2       2016-10-10 17:40:00
3       2016-10-10 17:40:01
4       2016-10-10 17:40:01
                ...        
99487   2017-06-24 11:06:00
99488   2017-06-24 11:07:02
99489   2017-06-24 11:12:00
99490   2017-06-24 11:17:01
99491   2017-06-24 11:17:02
Name: timeStamp, Length: 99492, dtype: datetime64[ns]

Lo mismo jala con:
- years
- months
- weeks
- days
- hours
- minutes
- seconds
- microseconds
- nanoseconds  


esto es muy similar a usar timedeltas! Sin embargo es diferente agregar un offset a agregar un cambio. Por ejemplo, si quiero mover mis fechas 10 meses hacia adelante queda claro que enero (1) se vuelve noviembre (11), pero si te digo que le sumes 10 meses a enero se vuelve difícil porque no sé cuánto dura un mes (28, 29, 30 ó 31 días?)  

Puesto de otra manera, Offset usa fechas relativas al calendario (sumar 1 semana literal se brinca una semana), mientras que Timedelta usa tiempo absoluto (sumar 1 semana es sumar exactamente 604800 segundos)... más adelante hay un ejemplo si no queda claro.

## Timedelta

In [54]:
fechas + pd.Timedelta(days = 10)

0       2015-12-20 17:40:00
1       2015-12-20 17:40:00
2       2015-12-20 17:40:00
3       2015-12-20 17:40:01
4       2015-12-20 17:40:01
                ...        
99487   2016-09-03 11:06:00
99488   2016-09-03 11:07:02
99489   2016-09-03 11:12:00
99490   2016-09-03 11:17:01
99491   2016-09-03 11:17:02
Name: timeStamp, Length: 99492, dtype: datetime64[ns]

In [55]:
fechas + pd.Timedelta(seconds = 60)

0       2015-12-10 17:41:00
1       2015-12-10 17:41:00
2       2015-12-10 17:41:00
3       2015-12-10 17:41:01
4       2015-12-10 17:41:01
                ...        
99487   2016-08-24 11:07:00
99488   2016-08-24 11:08:02
99489   2016-08-24 11:13:00
99490   2016-08-24 11:18:01
99491   2016-08-24 11:18:02
Name: timeStamp, Length: 99492, dtype: datetime64[ns]

**Offset vs timedelta**  
El 14 de marzo de 2021 habrá un cambio de horario.

In [56]:
dia_ejemplo = pd.Timestamp('14 march 2021 00:00', tz='US/Eastern')
dia_ejemplo

Timestamp('2021-03-14 00:00:00-0500', tz='US/Eastern')

Si agrego un día, Offset sólo recorre la fecha un día de calendario independientemente de si ese día va a 'durar' 23,24 ó 25 horas

In [57]:
dia_ejemplo + pd.DateOffset(days=1)

Timestamp('2021-03-15 00:00:00-0400', tz='US/Eastern')

Mientras que Timedelta suma 24 horas siempre

In [58]:
dia_ejemplo + pd.Timedelta(days=1)

Timestamp('2021-03-15 01:00:00-0400', tz='US/Eastern')

**En operaciones**

Si tomamos la diferencia absoluta (segundos) entre cada fecha y antecesora en el datafame obtenemos un Timedelta. De estos Timedeltas podemos pedir semanas, días, horas, minutos, segundos, us, ns. En este pequeño ejemplo vemos cuál fue el tiempo máximo que tuvimos entre eventos.

In [59]:
x = fechas.diff()
x

0                   NaT
1       0 days 00:00:00
2       0 days 00:00:00
3       0 days 00:00:01
4       0 days 00:00:00
              ...      
99487   0 days 00:03:58
99488   0 days 00:01:02
99489   0 days 00:04:58
99490   0 days 00:05:01
99491   0 days 00:00:01
Name: timeStamp, Length: 99492, dtype: timedelta64[ns]

El número es un poco difícil de apreciar en segundos

In [60]:
x.dt.seconds.nlargest()

55208    50656.0
3212     36481.0
2866     29941.0
48142     9300.0
48271     8280.0
Name: timeStamp, dtype: float64

Pero no son suficientes segundos para hacer un día

In [61]:
x.dt.days.nlargest()

1    0.0
2    0.0
3    0.0
4    0.0
5    0.0
Name: timeStamp, dtype: float64

Probar minutos es más complicado porque estos objetos sólo guardan días, segundos y sus fracciones, por lo que se tiene que hacer el cálculo manualmente. **No tengo idea de por qué es así**

In [62]:
xx = x.dt.total_seconds()
minutos, segundos = divmod(xx, 60)
minutos.nlargest()

55208    844.0
3212     608.0
2866     499.0
48142    155.0
48271    138.0
Name: timeStamp, dtype: float64

Misma lógica con horas

In [63]:
xx = x.dt.total_seconds()
minutos, segundos = divmod(xx, 3600)
minutos.nlargest()

55208    14.0
3212     10.0
2866      8.0
48142     2.0
48271     2.0
Name: timeStamp, dtype: float64

**tips de conversión:**  

segundos = Timedelta.total_seconds()  
horas, segundos = divmod(segundos, 3600)  
minutos, segundos = divmod(segundos, 60)

**Se usa total_seconds() en vez de seconds por si hay fracciones de segundo. 1us daría 0 seconds pero daría 0.000001 total seconds.**

## Rolling

Para hacer los famosos: moving averages. Reemplaza los valores por el promedio del actual valor y los n-1 anteriores.

Por ejemplo, aquí estamos viendo el promedio de 3 valores. Los primeros dos valores están vacío porque no tienen otra información con la cuál alimentarse.

In [64]:
df_fechas.rolling(3).mean()

,lat,lng,zip,e
fechas,,,,
2015-12-10 17:40:00,NaN,NaN,NaN,NaN
2015-12-10 17:40:00,NaN,NaN,NaN,NaN
2015-12-10 17:40:00,40.225706,-75.399316,19457.333333,1.0
2015-12-10 17:40:01,40.165132,-75.320056,19416.000000,1.0
2015-12-10 17:40:01,40.162942,-75.432946,NaN,1.0
...,...,...,...,...
2016-08-24 11:06:00,40.151899,-75.313988,19282.666667,1.0
2016-08-24 11:07:02,40.106356,-75.267722,19149.333333,1.0
2016-08-24 11:12:00,40.085091,-75.319092,19268.333333,1.0


# Grouper

In [66]:
df['fechas'] = fechas

Supongamos que quieres agrupar por fecha y zip

In [67]:
df.groupby(['fechas','zip']).mean()

lat        lng  e
fechas              zip                             
2015-12-10 17:40:00 19401.0  40.121182 -75.351975  1
                    19446.0  40.258061 -75.264680  1
                    19525.0  40.297876 -75.581294  1
2015-12-10 17:40:01 19044.0  40.182111 -75.127795  1
                    19401.0  40.116153 -75.343513  1
...                                ...        ... ..
2016-08-24 11:06:00 19401.0  40.132869 -75.333515  1
2016-08-24 11:07:02 19003.0  40.006974 -75.289080  1
2016-08-24 11:12:00 19401.0  40.115429 -75.334679  1
2016-08-24 11:17:01 19002.0  40.186431 -75.192555  1
2016-08-24 11:17:02 19446.0  40.207055 -75.317952  1

[84034 rows x 3 columns]

Ahora supongamos que quieres agrupar por fecha y zip, pero quieres resamplear esto a algo DIARIO! D:

In [68]:
df.groupby([pd.Grouper(key = 'fechas', freq = 'D'),'zip']).mean()

lat        lng  e
fechas     zip                             
2015-12-10 18041.0  40.414249 -75.509016  1
           18936.0  40.223778 -75.235399  1
           18964.0  40.298004 -75.330187  1
           18974.0  40.209337 -75.135266  1
           19002.0  40.179748 -75.222182  1
...                       ...        ... ..
2016-08-24 19446.0  40.231983 -75.314206  1
           19462.0  40.100344 -75.293955  1
           19464.0  40.255952 -75.640684  1
           19468.0  40.200388 -75.518098  1
           19473.0  40.222196 -75.406875  1

[13848 rows x 3 columns]